In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
import bz2
import json
import datetime
from datetime import datetime as dt
from collections import defaultdict
from itertools import product
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm as pbar
from utils import tick_delta, dollars_to_ticks
import pickle as p

### Todo

1. [ ] Extract timestamp per tick
1. [ ] Extract batl, batb
1. [ ] Extract volume off ticks
1. [ ] Extract bsp

In [4]:
def int_dict():
    return defaultdict(int)

def double_d():
    return defaultdict(dict)

In [5]:
def parse_entries(entries):
    batl = []
    batb = []
    batl_v = {}
    batb_v = {}
    for n in range(6):
        batl_v[n] = []
        batb_v[n] = []
            
    mrkts = []
    times = []
    sids = []
    tot_vols = []
    in_play = []
    mean_trd = []
    mrkt_inplay = {}
    tvs = defaultdict(int_dict)
    back_ladder = defaultdict(double_d)
    lay_ladder = defaultdict(double_d)
    trd_ladder = defaultdict(int_dict)

    market_defs = []
    market_defs_ids = []
    
    for entry in entries: # should be map
            ts = entry['pt']
            change_sids = []
            change_mids = []
            for rc in entry['mc']:
                if 'marketDefinition' in rc.keys():
                    play_mid = rc['id']
                    mrkt_inplay[play_mid] = rc['marketDefinition']['inPlay']
                    market_defs. append(rc['marketDefinition'])
                    market_defs_ids.append(rc['id'])
                if 'rc' in rc.keys():
                    mid = rc['id']
                    for change in rc['rc']:
                        sid = change['id']
                        change_sids.append(sid)
                        change_mids.append(mid)
                        keys = change.keys()
                        if 'atb' in keys:
                            for prc, vol in change['atb']:
                                back_ladder[mid][sid][prc] = vol
                        if 'atl' in keys:
                            for prc, vol in change['atl']:
                                lay_ladder[mid][sid][prc] = vol
                        if 'trd' in keys:
                            for prc, vol in change['trd']:
                                trd_ladder[sid][prc] += vol
            for sid, mid in zip(change_sids, change_mids):
                cur_batl = min([1000] + [k for k, v in lay_ladder[mid][sid].items() if v > 1])
                cur_batb = max([0] + [k for k, v in back_ladder[mid][sid].items() if v > 1])
                if cur_batl < 10 and cur_batb > 0:
                    tot_vol = 0
                    agg_vol = 0
                    for p, v in trd_ladder[sid].items():
                        agg_vol += float(p) * v
                        tot_vol += v
                    if tot_vol > 500:
                        mean_price = agg_vol / tot_vol if tot_vol > 1 else 0
                        mean_trd.append(mean_price)
                        times.append(ts)
                        sids.append(sid)
                        mrkts.append(mid)
                        batl.append(cur_batl)
                        batb.append(cur_batb)
                        for n in range(6):
                            adj_batl = tick_delta(cur_batl, n)
                            adj_batb = tick_delta(cur_batb, -n)
                            if adj_batl in lay_ladder[mid][sid].keys():
                                batl_v[n].append(lay_ladder[mid][sid][adj_batl])
                            else:
                                batl_v[n].append(None)
                            if adj_batb in back_ladder[mid][sid].keys():
                                batb_v[n].append(back_ladder[mid][sid][adj_batb])
                            else:
                                batb_v[n].append(None)
                        tot_vols.append(tot_vol)
                        in_play.append(mrkt_inplay[mid])

    df = pd.DataFrame({'timestamp': times, 
               'market_id': mrkts, 
               'sid': sids, 
               'batl': batl, 
               'batb': batb, 
               'batl_v': batl_v[0], 
               'batb_v': batb_v[0],
               'batl_v_1': batl_v[1], 
               'batb_v_1': batb_v[1], 
               'batl_v_2': batl_v[2], 
               'batb_v_2': batb_v[2], 
               'batl_v_3': batl_v[3], 
               'batb_v_3': batb_v[3], 
               'batl_v_4': batl_v[4], 
               'batb_v_4': batb_v[4], 
               'batl_v_5': batl_v[5], 
               'batb_v_5': batb_v[5], 
               'tot_vol': tot_vols,
                'mean_trd': mean_trd,
                'in_play': in_play})
    
    mrkt_def = pd.DataFrame(market_defs)
    mrkt_def['market_id'] = market_defs_ids
    
    return df, mrkt_def

In [6]:
run_id = dt.now().strftime('%m%d')
year = '2019'
month = 'Dec'
stem_path = '/Users/jake/Downloads/PRO 3/2019/Dec/'
gcloud_path = 'gs://h2o_temp/dev_sets/'

days = [x for x in os.listdir(stem_path) if len(x) <= 2]#[:2] # limit to first two days
final_paths = {}
for day in days:
    for fld in os.listdir(os.path.join(stem_path, day)):
        try:
            int(fld) # shit house validity check
            each_folder = os.path.join(stem_path, day, fld)
            for fn in os.listdir(each_folder):
                if '1.' in fn and fn.split('.')[-1] == 'bz2':
                    final_paths[fn] = os.path.join(stem_path, day, fld, fn)
        except:
            print(f'failed on {fld}')

In [7]:
dir_path = f'../data/prepared/{run_id}_{year}_{month}'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
    
count = len(final_paths.keys())
count

4622

In [8]:
keep_cols = ['market_id', 'eventId', 'eventTypeId', 'numberOfWinners','bettingType', 'marketType', 'marketTime', 'name', 'eventName']

dfs = []
defs = []
for k, v in pbar(final_paths.items(), '', count):
    try:
        fn = v
        if os.path.exists(os.path.join(dir_path, f'{k}.csv')):
            continue

        entries = []
        with bz2.open(fn, mode='r') as f:
            for line in f.readlines():
                entries.append(json.loads(line))

        df, mrkt_def = parse_entries(entries)
        runner_frame = pd.DataFrame(mrkt_def.runners.values[-1])
        runner_frame.rename({'id':'sid'}, axis=1, inplace=True)

        df = df.groupby(['timestamp','market_id', 'sid']).last().reset_index()
        df.timestamp = df.timestamp // 1000
        df.timestamp = df.timestamp.apply(dt.fromtimestamp)
        df.timestamp = pd.to_datetime(df.timestamp)
        df = df.merge(runner_frame[['sid', 'bsp']], on='sid', how='left')

        mrkt_def = mrkt_def[keep_cols]
        mrkt_def.drop_duplicates(inplace=True)
        mrkt_def = mrkt_def[(mrkt_def.marketType=='WIN')]
        mrkt_def.marketTime = pd.to_datetime(mrkt_def.marketTime)
        mrkt_def.marketTime += np.timedelta64(10,'h')

        df = df.merge(mrkt_def, on='market_id', how='inner')
        df['test_starttime'] = df.marketTime.dt.tz_convert(None)
        df['test_recordtime'] = df.test_starttime - np.timedelta64(20,'m')
        df = df[(df.timestamp < df.test_starttime) & (df.timestamp > df.test_recordtime)]

        df.to_csv(os.path.join(dir_path, f'{k}.csv'), index=False)
    
    except:
        continue